# Training tree-based models for time series forecasting

After working with the ARIMA model yesterday, today you will use the same data to fit a Random Forest (RF) and XGBoost model on it.

Throughout the seminar, we will use the following splits for training, validation, and testing. Make sure to keep the tests unseen until the final evaluation (information leakage):

- Training set: 2009-2013
- Validation set: 2014
- Test set: 2015-2016

Required python packages: pandas, numpy, matplotlib, scikit-learn, xgboost

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import xgboost as xgb

import os

In [2]:
train_file = os.path.join("..", "data", "interim", "training.csv")
val_file = os.path.join("..", "data", "interim", "validation.csv")
test_file = os.path.join("..", "data", "interim", "test.csv")

train = pd.read_csv(train_file)
val = pd.read_csv(val_file)
test = pd.read_csv(test_file)

## Task 1: Feature Engineering

For random forest and xgboost, engineer features for the flu-trends dataset by incorporating lagged values (e.g., 8 weeks); ensure that your feature engineering process does not introduce information from future data points into past records.

> For instance, use the past 8 weeks to predict the next week.X-columns: FluVisits_t-8, FluVisits_t-7, ..., FluVisits_t-1
>
> You can also predict more than one week ahead. Then you need to shift the next samples accordingly.X-columns: FluVisits_t-8, FluVisits_t-7, ..., FluVisits_t-1
Y-columns: FluVisits_t, FluVisits_t+1, FluVisits_t+2

```python
 def create_features(data: pd.DataFrame, target_column: str, feature_length: int = 8, prediction_length: int = 2) -> Tuple[np.ndarray, np.ndarray]:
    ...
    return X, y
```

In [3]:
def create_features(data: pd.DataFrame, target_column: str, feature_length: int = 8, prediction_length: int = 2) -> tuple[np.ndarray, np.ndarray]:
    """
    Create features and target arrays for time series forecasting.

    Args:
        data (pd.DataFrame): The input data containing the time series.
        target_column (str): The name of the target column to predict.
        feature_length (int, optional): The number of past observations to use as features. Defaults to 8.
        prediction_length (int, optional): The number of future observations to predict. Defaults to 2.

    Returns:
        tuple[np.ndarray, np.ndarray]: The feature and target arrays.
    """
    
    target_data = data[target_column].values
    for i in range(feature_length, len(data) - prediction_length + 1): # Iterate over rows from e.g. 8 to len(data)-2
        features = target_data[i-feature_length:i] # Get the last 8 values as features as np array
        target = target_data[i:i+prediction_length] # Get the next 2 values as target as np array
        # for first iteration, initialize X and y
        if i == feature_length:
            X = np.array([features])
            y = np.array(target)
        else: # Add new rows to ndarrays
            X = np.vstack([X, features])
            y = np.vstack([y, target])
    return X, y

In [4]:
X, y = create_features(train, target_column="FluVisits", prediction_length=2, feature_length=8)
X.shape, y.shape


((227, 8), (227, 2))

## Task 2: Fitting the models

Task 2.1: Fit a Random Forest model to the data. Evaluate the model using the test set.

In [5]:
rf = sk.ensemble.RandomForestRegressor(random_state=42)
rf.fit(X, y)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [6]:
X_test, y_test = create_features(test, target_column="FluVisits", prediction_length=2, feature_length=8)
print(f"Shape of test features and target:{(X_test.shape, y_test.shape)}")


y_pred = rf.predict(X_test)
print(f"Shape of y_pred: {y_pred.shape}")

Shape of test features and target:((69, 8), (69, 2))
Shape of y_pred: (69, 2)


In [7]:
R2_score = rf.score(X_test, y_test)
print(f"R2 score on test set: {R2_score}")

mse = sk.metrics.mean_squared_error(y_test, y_pred)
print(f"MSE on test set: {mse}")

mape = sk.metrics.mean_absolute_percentage_error(y_test, y_pred)
print(f"MAPE on test set: {mape}")

R2 score on test set: 0.6719457908495874
MSE on test set: 41498.65905579709
MAPE on test set: 0.22698652534581096


Task 2.2: Fit a XGBoost model to the data. Evaluate the model using the test set.

In [8]:
X_val, y_val = create_features(val, target_column="FluVisits", prediction_length=2, feature_length=8)

In [9]:
dtrain = xgb.DMatrix(X, label=y)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

model = xgb.XGBRegressor(random_state=42)

# Train the model
param = {'eval_metric': ['rmse', 'mape']} # Define parameters
evallist = [(dtrain, 'train'), (dval, 'eval')]
bst = xgb.train(param, dtrain, 100, evals=evallist)

[0]	train-rmse:445.42906	train-mape:4.97791	eval-rmse:670.80139	eval-mape:4.98954
[1]	train-rmse:334.17068	train-mape:3.54966	eval-rmse:583.54463	eval-mape:3.57967
[2]	train-rmse:254.74717	train-mape:2.53722	eval-rmse:512.49826	eval-mape:2.58164
[3]	train-rmse:196.72781	train-mape:1.81389	eval-rmse:468.52465	eval-mape:1.86650
[4]	train-rmse:152.82028	train-mape:1.30396	eval-rmse:431.39683	eval-mape:1.37016
[5]	train-rmse:120.13680	train-mape:0.94185	eval-rmse:405.61856	eval-mape:1.01817
[6]	train-rmse:95.32056	train-mape:0.68138	eval-rmse:385.02724	eval-mape:0.76436
[7]	train-rmse:77.12253	train-mape:0.49445	eval-rmse:371.15159	eval-mape:0.57892
[8]	train-rmse:62.01500	train-mape:0.36314	eval-rmse:359.78903	eval-mape:0.46536
[9]	train-rmse:50.33594	train-mape:0.26832	eval-rmse:351.84744	eval-mape:0.38079
[10]	train-rmse:40.99286	train-mape:0.20164	eval-rmse:346.49216	eval-mape:0.32380
[11]	train-rmse:33.75935	train-mape:0.15944	eval-rmse:341.35092	eval-mape:0.28803
[12]	train-rmse:27.9

In [10]:
y_pred = bst.predict(dtest)
#xgb.to_graphviz(bst)

Describe the performance of the models and compare them in a few sentences. How do they perform in comparison to each other?

## Task 3: Hyperparameter tuning via grid search

Tune the hyperparameters of the Random Forest and XGBoost models using grid search. Focus on the following hyperparameters:

Random Forest:
- n_estimators
- max_depth
- ...

XGBoost:
- gamma
- max_depth
- eta
- ...

Optimize the hyperparameters using the validation set and the refit the models using the best hyperparameters on the entire training set and evaluate them on the test set.

In [31]:
rf_param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20, 30],
    'criterion': ['squared_error', 'absolute_error', 'poisson'],
    'max_features': ['sqrt', 'log2', None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

In [32]:
import itertools
keys, values = zip(*rf_param_grid.items())
permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

In [ ]:
best_R2 = -np.inf
best_param = None
for param in permutations_dicts:
    print(param)
    rf = sk.ensemble.RandomForestRegressor(random_state=42, **param)
    rf.fit(X_test, y_test)
    y_val_pred = rf.predict(X_val)
    R2_score = rf.score(X_val, y_val)
    if R2_score > best_R2:
        best_R2 = R2_score
        best_param = param
    print(f"R2 score on val set: {R2_score}")


{'n_estimators': 100, 'max_depth': None, 'criterion': 'squared_error', 'max_features': 'sqrt', 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': True}
R2 score on val set: 0.4540385073519166
{'n_estimators': 100, 'max_depth': None, 'criterion': 'squared_error', 'max_features': 'sqrt', 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}
R2 score on val set: 0.49857253104164506
{'n_estimators': 100, 'max_depth': None, 'criterion': 'squared_error', 'max_features': 'sqrt', 'min_samples_split': 2, 'min_samples_leaf': 2, 'bootstrap': True}
R2 score on val set: 0.4209674156964229
{'n_estimators': 100, 'max_depth': None, 'criterion': 'squared_error', 'max_features': 'sqrt', 'min_samples_split': 2, 'min_samples_leaf': 2, 'bootstrap': False}
R2 score on val set: 0.46430894923671867
{'n_estimators': 100, 'max_depth': None, 'criterion': 'squared_error', 'max_features': 'sqrt', 'min_samples_split': 2, 'min_samples_leaf': 4, 'bootstrap': True}
R2 score on val set: 0.34874485

In [ ]:
best_R2, best_param

(0.5770394101448187, {'n_estimators': 200, 'max_depth': None})

## Task 4: Time series cross-validation

Implement a time series cross-validation strategy to optimize the hyperparameters of the models. Except for the cross-validation strategy, the procedure is the same as in Task 3. You only need to define different training and validation sets before performing the grid search. In the end, you take the parameters that performed best across all folds and refit the models on the entire training set and evaluate them on the test set.

In [11]:
# Time series function will do split for us.

# Optional Task:
- Take a look which other features could be good predictors for the flu visits. How does including additional features impact the overall model performance?